# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig

# -------------------------------------------------------
# Global test constants
# -------------------------------------------------------

USER_ID = "demo-user-planner"
SESSION_ID = "sess_planner_only_001"

# -------------------------------------------------------
# Build LLM adapter and RuntimeConfig
# IMPORTANT: Replace build_llm_adapter() with your actual builder.
# This must be the same adapter/config used in your runtime.
# -------------------------------------------------------

llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=True,
    enable_websearch=True,
    tools_mode="auto",
    enable_user_longterm_memory=True,    
)

# Hard-coded intent

In [4]:
from intergrax.runtime.drop_in_knowledge_mode.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.drop_in_knowledge_mode.planning.stepplan_models import EngineHints, PlanIntent, StepAction

sp = StepPlanner(StepPlannerConfig())

# TEST 2: freshness -> websearch
plan = sp.build_plan(
    user_message="What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates.",
    engine_hints=EngineHints(
        enable_websearch=True,
        enable_ltm=True,
        enable_rag=False,
        enable_tools=False,
        intent=PlanIntent.FRESHNESS,
    ),
)
assert plan.intent == PlanIntent.FRESHNESS
assert plan.steps[0].action == StepAction.USE_WEBSEARCH
print("OK freshness routing:", plan.intent, [s.action.value for s in plan.steps])

# TEST 3: project -> ltm
plan = sp.build_plan(
    user_message="StepPlanner vs EnginePlanner integration in Intergrax. What should own intent?",
    engine_hints=EngineHints(
        enable_websearch=True,
        enable_ltm=True,
        enable_rag=False,
        enable_tools=False,
        intent=PlanIntent.PROJECT_ARCHITECTURE,
    ),
)
assert plan.intent == PlanIntent.PROJECT_ARCHITECTURE
assert plan.steps[0].action == StepAction.USE_USER_LONGTERM_MEMORY_SEARCH
print("OK project routing:", plan.intent, [s.action.value for s in plan.steps])

OK freshness routing: PlanIntent.FRESHNESS ['USE_WEBSEARCH', 'SYNTHESIZE_DRAFT', 'VERIFY_ANSWER', 'FINALIZE_ANSWER']
OK project routing: PlanIntent.PROJECT_ARCHITECTURE ['USE_USER_LONGTERM_MEMORY_SEARCH', 'SYNTHESIZE_DRAFT', 'VERIFY_ANSWER', 'FINALIZE_ANSWER']


In [5]:
from intergrax.runtime.drop_in_knowledge_mode.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.drop_in_knowledge_mode.planning.stepplan_models import EngineHints


sp = StepPlanner(StepPlannerConfig())

tests = [
    "Explain how to implement an async retry strategy in Python for API calls.",
    "What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates.",
    "In my Intergrax runtime, should StepPlanner be separate from EnginePlanner? Give recommendation based on our architecture decisions.",
]

for i, q in enumerate(tests, 1):
    plan = sp.build_plan(user_message=q, engine_hints=EngineHints(
        enable_websearch=True,
        enable_ltm=True,
        enable_rag=False,
        enable_tools=False,
    ))
    print("=" * 80)
    print(f"TEST {i}: {q}")
    print("plan_id:", plan.plan_id)
    print("mode:", plan.mode)
    print("intent:", plan.intent)
    print("steps:", [s.action for s in plan.steps])
    print("last:", plan.steps[-1].action)


TEST 1: Explain how to implement an async retry strategy in Python for API calls.
plan_id: stepplan-001
mode: PlanMode.EXECUTE
intent: PlanIntent.GENERIC
steps: [<StepAction.SYNTHESIZE_DRAFT: 'SYNTHESIZE_DRAFT'>, <StepAction.VERIFY_ANSWER: 'VERIFY_ANSWER'>, <StepAction.FINALIZE_ANSWER: 'FINALIZE_ANSWER'>]
last: StepAction.FINALIZE_ANSWER
TEST 2: What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates.
plan_id: stepplan-001
mode: PlanMode.EXECUTE
intent: PlanIntent.GENERIC
steps: [<StepAction.SYNTHESIZE_DRAFT: 'SYNTHESIZE_DRAFT'>, <StepAction.VERIFY_ANSWER: 'VERIFY_ANSWER'>, <StepAction.FINALIZE_ANSWER: 'FINALIZE_ANSWER'>]
last: StepAction.FINALIZE_ANSWER
TEST 3: In my Intergrax runtime, should StepPlanner be separate from EnginePlanner? Give recommendation based on our architecture decisions.
plan_id: stepplan-001
mode: PlanMode.EXECUTE
intent: PlanIntent.GENERIC
steps: [<StepAction.SYNTHESIZE_DRAFT: 'SYNTHESIZE_DRAFT'>, <StepAction.VERIFY_ANS

OK freshness routing: PlanIntent.FRESHNESS ['USE_WEBSEARCH', 'SYNTHESIZE_DRAFT', 'VERIFY_ANSWER', 'FINALIZE_ANSWER']
OK project routing: PlanIntent.PROJECT_ARCHITECTURE ['USE_USER_LONGTERM_MEMORY_SEARCH', 'SYNTHESIZE_DRAFT', 'VERIFY_ANSWER', 'FINALIZE_ANSWER']
